# 02. Train Model

ToDo:
    * Create new feature (999)
    * 1.11.5.3. Using the VotingClassifier with GridSearch
    * bin
    * other cat encoding
    * cross valid

Done:
    * Double pos (not improve)

Changelog:
    * maj vot
    * use reciprocal
    * use kbest

In [21]:
VERSION = '180612_v2'

In [22]:
# all imports
import sys
sys.path.insert(0, '../../src/')

import pandas as pd
import os
import transformers
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from transformers import (ModelTransformer, DataFrameColumnExtractor, ToDictTransformer)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

#evaluators
#gridcv

### Read data

In [23]:
DATA_DIR = '../../data/raw/'
TRAIN_DATASET_PATH = os.path.join(DATA_DIR, 'train.csv')
TEST_DATASET_PATH = os.path.join(DATA_DIR, 'test.csv')

In [24]:
df_train = pd.read_csv(TRAIN_DATASET_PATH, encoding='utf-8')
df_test = pd.read_csv(TEST_DATASET_PATH, encoding='utf-8')

### Train/Test Data

In [25]:
X = df_train.drop(['y', 'id'], axis=1)
y = df_train['y']

In [26]:
X_train, X_test, y_train, y_test = cv.train_test_split(X,
                                                       y,
                                                       test_size=0.25,
                                                       random_state=42)

In [27]:
# X_train_yes = X_train[y_train == 1]
# y_train_yes = y_train[y_train == 1]

# X_train = pd.concat([X_train, X_train_yes])
# y_train = pd.concat([y_train, y_train_yes])

### Train model

In [28]:
df_train[df_train['pdays'] == 999].count()

id                34906
y                 34906
age               34906
job               34906
marital           34906
education         34906
default           34906
housing           34906
loan              34906
contact           34906
month             34906
day_of_week       34906
campaign          34906
pdays             34906
previous          34906
poutcome          34906
emp.var.rate      34906
cons.price.idx    34906
cons.conf.idx     34906
euribor3m         34906
nr.employed       34906
dtype: int64

In [29]:
CATEGORICAL_FEATURES = [
    'job',
    'marital',
    'education',
    'default',
    'housing',
    'loan',
    'contact',
    'poutcome',
    'month',
    'day_of_week'
]

NUMERIC_FEATURES = [                    
    'age',
#     'campaign',
    'pdays',
#     'previous',
    'emp.var.rate',
    'cons.price.idx',
    'cons.conf.idx',
    'euribor3m',
    'nr.employed'
]

TO_APPLY_LOG = [
]

TO_APPLY_CUBE_ROOT = [

]

TO_APPLY_RECIPROCAL = [
    'campaign',
    'previous'
]

TO_BIN = [

]

TRAINING_FEATURES = NUMERIC_FEATURES + CATEGORICAL_FEATURES
ALL_COLUMNS = TRAINING_FEATURES + ['id', 'y']

In [30]:
transformer_pipeline = make_pipeline(
    make_pipeline(

        make_union(
            make_pipeline(
                make_union(
                    make_pipeline(
                        DataFrameColumnExtractor(NUMERIC_FEATURES),
                    ),
                    make_pipeline(
                        DataFrameColumnExtractor(TO_APPLY_RECIPROCAL),
                        FunctionTransformer(transformers.sum_1),
                        FunctionTransformer(transformers.apply_reciprocal),
                    )
                ),
                StandardScaler(),
            ),
            make_pipeline(
                DataFrameColumnExtractor(CATEGORICAL_FEATURES),
                ToDictTransformer(),
                DictVectorizer(sparse=False)
            ),

        ),
        SelectKBest(f_classif, k=55)
    )
)

# DataFrameColumnExtractor(to_bin),
# FunctionTransformer(transformers.bin_data, kw_args={'columns': to_bin, 'bins': bins}),

In [31]:
# clf = VotingClassifier(estimators=[
#     ('gbc', GradientBoostingClassifier(random_state=42, max_depth=4)), 
#     ('xgb', XGBClassifier(random_state=42)), 
#     ('lr', LogisticRegression(random_state=42)),
#     ('rf', RandomForestClassifier(random_state=42)),
#     ('gau', GaussianNB())
# ], voting='soft')

predictor_pipeline = make_pipeline(
    VotingClassifier(estimators=[
        ('gbc', GradientBoostingClassifier(random_state=42, max_depth=4)), 
        ('xgb', XGBClassifier(random_state=42)), 
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('gau', GaussianNB()),
        ('nn', MLPClassifier(random_state=42, solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2))),
#         ('svm', SVC(random_state=42, probability=True)),
    ], voting='soft')
)

#      make_union(
#          ModelTransformer(SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#             decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#             max_iter=-1, probability=False, random_state=None, shrinking=True,
#             tol=0.001, verbose=False)),
#          FunctionTransformer(transformers.all_columns)
#      ),

In [32]:
pipeline= Pipeline([
    ('features', transformer_pipeline),
    ('predictor', predictor_pipeline)
])

In [33]:
pipeline.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


Pipeline(steps=[('features', Pipeline(steps=[('pipeline', Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(steps=[('dataframecolumnextractor', DataFrameColum...0.1, verbose=False,
       warm_start=False))],
         n_jobs=1, voting='soft', weights=None))]))])

### Evaluate model

In [34]:
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [35]:
print(metrics.classification_report(y_test, y_pred))

print("Precision: "+ str(metrics.precision_score(y_test, y_pred)))
print("Recall: "+ str(metrics.recall_score(y_test, y_pred)))
print("F1 Score: "+ str(metrics.f1_score(y_test, y_pred, average='weighted')))

             precision    recall  f1-score   support

          0       0.92      0.97      0.95      8077
          1       0.61      0.35      0.45       985

avg / total       0.89      0.91      0.89      9062

Precision: 0.6141592920353982
Recall: 0.3522842639593909
F1 Score: 0.8939386128529604


### Generate submission file

In [36]:
DATA_DIR = '../../data/raw/'
TRAIN_DATASET_PATH = os.path.join(DATA_DIR, 'train.csv')
TEST_DATASET_PATH = os.path.join(DATA_DIR, 'test.csv')

In [37]:
df_train = pd.read_csv(TRAIN_DATASET_PATH, encoding='utf-8')
df_test = pd.read_csv(TEST_DATASET_PATH, encoding='utf-8')

In [38]:
X_train = df_train.drop(['y', 'id'], axis=1)
y_train = df_train['y']

X_test = df_test.drop(['id'], axis=1)

In [39]:
# X_train_yes = X_train[y_train == 1]
# y_train_yes = y_train[y_train == 1]

# X_train = pd.concat([X_train, X_train_yes])
# y_train = pd.concat([y_train, y_train_yes])

In [40]:
pipeline.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


Pipeline(steps=[('features', Pipeline(steps=[('pipeline', Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(steps=[('dataframecolumnextractor', DataFrameColum...0.1, verbose=False,
       warm_start=False))],
         n_jobs=1, voting='soft', weights=None))]))])

In [41]:
pred_submission = pipeline.predict(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [42]:
ids_submission = df_test['id']

In [43]:
# Save submission file
SUBMISSION_DATA_DIR = '../../data/submission/'
SUBMISSION_FILE_PATH = os.path.join(SUBMISSION_DATA_DIR, VERSION+'.csv')

df_submission = pd.DataFrame({'id':ids_submission, 'y':pred_submission})
df_submission.to_csv(SUBMISSION_FILE_PATH, index=False)

In [44]:
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9062 entries, 0 to 9061
Data columns (total 2 columns):
id    9062 non-null int64
y     9062 non-null int64
dtypes: int64(2)
memory usage: 141.7 KB
